In [1]:
# check version
from pycaret.utils import version
version()

'2.3.6'

In [2]:
import pandas as pd
import numpy as np
import warnings
import optuna
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [4]:
df=pd.read_csv(r"C:\Users\jloom\Desktop\hub_projects\healthcare_covid19\PyCaret Mahine Learning pred\healthcare_pycaret.csv")

In [ ]:
#df.drop(columns=['Unnamed: 0'],inplace=True)

In [5]:
df

,Available Extra Rooms in Hospital,Bed Grade,Visitors with Patient,Admission_Deposit,Stay,Hospital_code_1,Hospital_code_2,Hospital_code_3,Hospital_code_4,Hospital_code_5,Hospital_code_6,Hospital_code_7,Hospital_code_8,Hospital_code_9,Hospital_code_10,Hospital_code_11,Hospital_code_12,Hospital_code_13,Hospital_code_14,Hospital_code_15,Hospital_code_16,Hospital_code_17,Hospital_code_18,Hospital_code_19,Hospital_code_20,Hospital_code_21,Hospital_code_22,Hospital_code_23,Hospital_code_24,Hospital_code_25,Hospital_code_26,Hospital_code_27,Hospital_code_28,Hospital_code_29,Hospital_code_30,Hospital_code_31,Hospital_code_32,Hospital_type_code_a,Hospital_type_code_b,Hospital_type_code_c,Hospital_type_code_d,Hospital_type_code_e,Hospital_type_code_f,Hospital_type_code_g,City_Code_Hospital_1,City_Code_Hospital_2,City_Code_Hospital_3,City_Code_Hospital_4,City_Code_Hospital_5,City_Code_Hospital_6,City_Code_Hospital_7,City_Code_Hospital_9,City_Code_Hospital_10,City_Code_Hospital_11,City_Code_Hospital_13,Hospital_region_code_X,Hospital_region_code_Y,Hospital_region_code_Z,Department_TB & Chest disease,Department_anesthesia,Department_gynecology,Department_radiotherapy,Department_surgery,Ward_Type_P,Ward_Type_Q,Ward_Type_R,Ward_Type_S,Ward_Type_T,Ward_Type_U,Ward_Facility_Code_A,Ward_Facility_Code_B,Ward_Facility_Code_C,Ward_Facility_Code_D,Ward_Facility_Code_E,Ward_Facility_Code_F,City_Code_Patient_1.0,City_Code_Patient_2.0,City_Code_Patient_3.0,City_Code_Patient_4.0,City_Code_Patient_5.0,City_Code_Patient_6.0,City_Code_Patient_7.0,City_Code_Patient_8.0,City_Code_Patient_9.0,City_Code_Patient_10.0,City_Code_Patient_11.0,City_Code_Patient_12.0,City_Code_Patient_13.0,City_Code_Patient_14.0,City_Code_Patient_15.0,City_Code_Patient_16.0,City_Code_Patient_18.0,City_Code_Patient_19.0,City_Code_Patient_20.0,City_Code_Patient_21.0,City_Code_Patient_22.0,City_Code_Patient_23.0,City_Code_Patient_24.0,City_Code_Patient_25.0,City_Code_Patient_26.0,City_Code_Patient_27.0,City_Code_Patient_28.0,City_Code_Patient_29.0,City_Code_Patient_30.0,City_Code_Patient_31.0,City_Code_Patient_32.0,City_Code_Patient_33.0,City_Code_Patient_34.0,City_Code_Patient_35.0,City_Code_Patient_36.0,City_Code_Patient_37.0,City_Code_Patient_38.0,Type of Admission_Emergency,Type of Admission_Trauma,Type of Admission_Urgent,Severity of Illness_Extreme,Severity of Illness_Minor,Severity of Illness_Moderate,Age_0-10,Age_11-20,Age_21-30,Age_31-40,Age_41-50,Age_51-60,Age_61-70,Age_71-80,Age_81-90,Age_91-100
0,3,2.0,2,4911.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
1,2,2.0,2,5954.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0
2,2,2.0,2,4745.0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0
3,2,2.0,2,7272.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0
4,2,2.0,2,5558.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [6]:
df.columns

Index(['Available Extra Rooms in Hospital', 'Bed Grade',
       'Visitors with Patient', 'Admission_Deposit', 'Stay', 'Hospital_code_1',
       'Hospital_code_2', 'Hospital_code_3', 'Hospital_code_4',
       'Hospital_code_5',
       ...
       'Age_0-10', 'Age_11-20', 'Age_21-30', 'Age_31-40', 'Age_41-50',
       'Age_51-60', 'Age_61-70', 'Age_71-80', 'Age_81-90', 'Age_91-100'],
      dtype='object', length=128)

In [7]:
from pycaret.classification import *

In [8]:
# Setting up the classifier
# Pass the complete dataset as data and the featured to be predicted as target
clf=setup(data=df,
          target='Stay',
          remove_multicollinearity=True,
          fix_imbalance=True,
          data_split_stratify= True)

,Description,Value
0,session_id,1703
1,Target,Stay
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(318438, 128)"
5,Missing Values,0
6,Numeric Features,3
7,Categorical Features,124
8,Ordinal Features,0
9,High Cardinality Features,0


### we choose the best model

In [9]:
models()

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron....,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


In [10]:
# This model will be used to compare all the model along with the cross validation
best_model=compare_models(include=["lightgbm","gbc"])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.7298,0.7748,0.8805,0.7603,0.8160,0.3176,0.3301,10.3620
gbc,Gradient Boosting Classifier,0.7159,0.7575,0.8486,0.7613,0.8026,0.3015,0.3072,78.4550


### create the model

In [11]:
#light GBM Model
lightgbm = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7296,0.7704,0.8811,0.7599,0.8160,0.3166,0.3293
1,0.7295,0.7725,0.8800,0.7603,0.8158,0.3173,0.3296
2,0.7346,0.7764,0.8804,0.7651,0.8187,0.3330,0.3446
3,0.7246,0.7735,0.8770,0.7568,0.8125,0.3042,0.3162
4,0.7292,0.7749,0.8798,0.7601,0.8156,0.3164,0.3287
5,0.7305,0.7770,0.8821,0.7603,0.8167,0.3186,0.3314
6,0.7301,0.7741,0.8799,0.7608,0.8161,0.3190,0.3312
7,0.7294,0.7779,0.8835,0.7586,0.8163,0.3140,0.3275
8,0.7283,0.7733,0.8784,0.7598,0.8148,0.3148,0.3268
9,0.7319,0.7776,0.8829,0.7613,0.8176,0.3224,0.3353


In [12]:
lightgbm

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=1703, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

### Tuning the hyperparametes for better performance

In [13]:
param_grid = {
    'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)],
    'colsample_bytree': [0.4,0.6,0.7, 0.8],
    'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)],
    'num_leaves': [int(x) for x in np.linspace(start = 20, stop = 3000, num = 20)],
    'reg_alpha': [1.1, 1.2, 1.3, 2, 3, 4, 5, 20, 50],
    'reg_lambda': [1.1, 1.2, 1.3, 2 , 3, 4, 5, 20, 50],
    'min_split_gain': [0.3, 0.4],
    "max_bin": [200, 300],
    "min_data_in_leaf": [int(x) for x in np.linspace(start = 200, stop = 10000, num =100)],
    "min_gain_to_split": [0, 15],
    'subsample': [0.7, 0.8, 0.9],
    'subsample_freq': [20],
    "learning_rate": [0.01,0.001,0.1,0.3],
    "lambda_l1": [int(x) for x in np.linspace(0,100,num = 5)],
    "lambda_l2": [int(x) for x in np.linspace(0,100,num = 5)],
    'reg_alpha': [0, 1, 2, 5, 7, 10, 50, 100],
    'reg_lambda': [0, 1, 5, 10, 20, 50, 100],
    "bagging_fraction": [0.2, 0.4,0.6,0.8,0.95],
    "feature_fraction": [0.2, 0.4,0.6,0.8,0.95],
    "bagging_freq": [1]}

In [14]:
#Hyper params tuning via tune_model
tuned_lightgbm  = tune_model(lightgbm,
                             #search_library = 'optuna'
                             search_algorithm = "random",  #"random"
                             custom_grid = param_grid,
                             early_stopping=True)
                             #early_stopping_round=10)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7223,0.7615,0.8559,0.7643,0.8075,0.3150,0.3216
1,0.7193,0.7615,0.8591,0.7598,0.8064,0.3031,0.3107
2,0.7220,0.7617,0.8574,0.7632,0.8076,0.3126,0.3196
3,0.7182,0.7600,0.8557,0.7603,0.8052,0.3026,0.3096
4,0.7200,0.7624,0.8566,0.7616,0.8063,0.3072,0.3142
5,0.7217,0.7635,0.8604,0.7616,0.8080,0.3094,0.3171
6,0.7190,0.7624,0.8512,0.7632,0.8048,0.3087,0.3146
7,0.7255,0.7692,0.8603,0.7654,0.8101,0.3212,0.3284
8,0.7199,0.7602,0.8554,0.7621,0.8061,0.3081,0.3148
9,0.7222,0.7652,0.8571,0.7636,0.8077,0.3137,0.3206


In [15]:
tuned_lightgbm

LGBMClassifier(bagging_fraction=0.6, bagging_freq=1, boosting_type='gbdt',
               class_weight=None, colsample_bytree=0.7, feature_fraction=0.95,
               importance_type='split', lambda_l1=25, lambda_l2=100,
               learning_rate=0.3, max_bin=200, max_depth=50,
               min_child_samples=20, min_child_weight=0.001,
               min_data_in_leaf=2872, min_gain_to_split=15, min_split_gain=0.4,
               n_estimators=944, n_jobs=-1, num_leaves=647, objective=None,
               random_state=1703, reg_alpha=10, reg_lambda=20, silent='warn',
               subsample=0.8, subsample_for_bin=200000, subsample_freq=20)

In [16]:
evaluate_model(tuned_lightgbm)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [ ]:
interpret_model(tuned_lightgbm)

In [ ]:
#predict_new = predict_model(tuned_lgbm, data=df)
#predict_new.head()

In [ ]:
plot_model(tuned_lightgbm, plot = 'auc')

In [ ]:
plot_model(tuned_lightgbm, plot='confusion_matrix')

In [ ]:
#plot_model(tuned_lightgbm, plot='learning')

In [ ]:
plot_model(tuned_lightgbm, plot='boundary')